# How can machine learning be applied to predict siteviability scores for offshore basins?

In my second subquestion, I use a new dataset of potential offshore injection sites across the Gulf of Mexico from Nassabeh et al. (2024). 

# Import Necessary Packages

In [12]:
import pandas as pd
import numpy as np
import ibis
from ibis import _
import ibis.selectors as s
from plotnine import *
import pandas as pd
con = ibis.duckdb.connect()
from sklearn.preprocessing import OneHotEncoder

# Read in Data from Nassabeh et al. (2024)

For this dataset, I use ibis with Duckdb to read in the data instead of pandas. This is due to the complexity of the data, and ibis handles this better internally. I also use ibis to accomodate the larger size of data I am working with.

In [9]:
nassabeh = con.read_csv('nassabeh_data.csv', skip =3)

In [10]:
nassabeh.head().execute()

,Grid FID,Grid ID,Longitude\n(degrees),Latitude\n(degrees),CO2 Source ID (tied to NATCARB dataset)*,Distance (km),CO2 Source ID (tied to NATCARB dataset),Distance (km)_1,CO2 Source ID (tied to NATCARB dataset)_1,Distance (km)_2,...,Sum of Reservoir Quality W/O Depth (feet),New Injectivity,S1,S2,S3,S4,S1_1,S2_1,S3_1,S4_1
0,0,36663,-96.964053,26.000342,1671,56.265652,860,39.710421,286,127.205047,...,9.1271,12.3958,57.746243,NaN,38.572100,NaN,25-50%,None,25-50%,None
1,1,36668,-97.035537,26.071826,1671,50.940346,860,34.179635,286,117.880692,...,1.5259,1.7557,30.571945,NaN,16.120983,NaN,0-25%,None,0-25%,None
2,2,36669,-96.964053,26.071826,1671,57.783507,860,41.034622,286,124.718795,...,9.1271,12.3958,57.670181,NaN,38.559063,NaN,25-50%,None,25-50%,None
3,3,36670,-97.035537,26.143310,1671,53.574517,860,37.428918,286,115.740856,...,9.1271,12.3958,32.312887,NaN,14.222778,NaN,0-25%,None,0-25%,None
4,4,36671,-96.964053,26.143310,1671,60.118581,860,43.774470,286,122.694148,...,9.1271,12.3958,57.596115,NaN,38.506005,NaN,25-50%,None,25-50%,None


In [11]:
nassabeh.columns

('Grid FID',
 'Grid ID',
 'Longitude\n(degrees)',
 'Latitude\n(degrees)',
 'CO2 Source ID (tied to NATCARB dataset)*',
 'Distance (km)',
 'CO2 Source ID (tied to NATCARB dataset)',
 'Distance (km)_1',
 'CO2 Source ID (tied to NATCARB dataset)_1',
 'Distance (km)_2',
 'Caisson Platform',
 'Fixed Platform',
 'Well Protector Platform',
 'Pipeline',
 'Fault',
 'Salt dome',
 'Shipping Route (w/5 MI Buffer)',
 'Shipping Route (w/10 mi Buffer)',
 'Fault_1',
 'Active Wells',
 'P&A Wells',
 'Caisson Platform_1',
 'Major Platform',
 'Well Protector Platform_1',
 'Pipeline_1',
 'Salt dome_1',
 'EOR Wells',
 'Shipping Route (w/10 mi Buffer)_1',
 'Number of plays',
 '(SUM) Injectivity W Depth - (kh)',
 '(SUM) Quality W/O Depth - ([h x E]/[1,000 feet to reservoir bottom])',
 'Capacity per Grid - (million tonnes)',
 'Water Depth - (feet)',
 'Average Calculated Water Depth - (feet)',
 '(SUM) Hydrocarbon Potential W/O Depth - ([h x So]/[1,000 feet to reservoir bottom])',
 'Recoverable Oil (barrels)/acr